In [1]:
import requests
from urllib.parse import quote
import pandas as pd

In [2]:
API_KEY = "RGAPI-85a3927d-f264-42ce-85b8-356382fedea0"
GAME_NAME = "X8Scorpio8X"
TAG_LINE = "NA1"  # do not include "#"
REGION = "americas"

def get_puuid_from_riot_id(game_name, tag_line, api_key, region):
    # URL-encode the game name and tag line
    game_name_encoded = quote(game_name)
    tag_line_encoded = quote(tag_line)
    
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name_encoded}/{tag_line_encoded}"
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data["puuid"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, API_KEY, REGION)
if puuid:
    print("PUUID:", puuid)


PUUID: vrfSU4Nh3GYoxHi2hLXg_yOczDqdE8WulZzWDLl2RZu6wtmH-e8i629HTEVbu1Uej1CpJZ3JIfXu3w


In [3]:
PUUID = puuid
REGION = "americas"  # match routing region

def get_match_ids(puuid, api_key, region, count=5):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count  # number of matches to retrieve (max 100 per call)
    }
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # list of match IDs
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
match_ids = get_match_ids(PUUID, API_KEY, REGION)
if match_ids:
    print("Recent Match IDs:", match_ids)


Recent Match IDs: ['NA1_5279161872', 'NA1_5279058507', 'NA1_5277980151', 'NA1_5277948017', 'NA1_5277488964']


In [4]:
def parse_match_data(match_data):
    if match_data['info']['queueId'] != 420:  # Only solo/duo ranked games
        return []

    teams = match_data['info']['teams']
    participants = match_data['info']['participants']

    # Split participants by teamId
    team1_id = teams[0]['teamId']
    team2_id = teams[1]['teamId']
    team1_players = [p for p in participants if p['teamId'] == team1_id]
    team2_players = [p for p in participants if p['teamId'] == team2_id]

    def team_stats(team, players, other_players):
        def get_ping_sum(key):
            return sum(p.get(key, 0) for p in players)

        return {
            'matchId': match_data['metadata']['matchId'],
            'teamId': team['teamId'],
            'win': team['win'],
            'baronKills': team['objectives']['baron']['kills'],
            'dragonKills': team['objectives']['dragon']['kills'],
            'riftHeraldKills': team['objectives']['riftHerald']['kills'],
            'totalGold': sum(p['goldEarned'] for p in players),
            'goldSpent': sum(p['goldSpent'] for p in players),
            'totalKills': sum(p['kills'] for p in players),
            'totalDeaths': sum(p['deaths'] for p in players),
            'totalTurretsTaken': sum(p.get('challenges', {}).get('turretTakedowns', 0) for p in players),
            'totalInhibitorsTaken': sum(p.get('challenges', {}).get('inhibitorTakedowns', 0) for p in players),
            'totalEliteMonstersTaken': sum(p.get('challenges', {}).get('eliteMonsterKills', 0) for p in players),
            'averageChampionExp': sum(p['champExperience'] for p in players) / len(players),
            'wardsPlaced': sum(p['wardsPlaced'] for p in players),
            'goldDiff': sum(p['goldEarned'] for p in players) - sum(p['goldEarned'] for p in other_players),
            'allInPings': get_ping_sum('allInPings'),
            'assistMePings': get_ping_sum('assistMePings'),
            'commandPings': get_ping_sum('commandPings'),
            'enemyMissingPings': get_ping_sum('enemyMissingPings'),
            'enemyVisionPings': get_ping_sum('enemyVisionPings'),
            'holdPings': get_ping_sum('holdPings'),
            'getBackPings': get_ping_sum('getBackPings'),
            'totalPings': (
                get_ping_sum('allInPings') +
                get_ping_sum('assistMePings') +
                get_ping_sum('commandPings') +
                get_ping_sum('enemyMissingPings') +
                get_ping_sum('enemyVisionPings') +
                get_ping_sum('holdPings') +
                get_ping_sum('getBackPings')
            )
        }


    return [team_stats(teams[0], team1_players, team2_players),
            team_stats(teams[1], team2_players, team1_players)]


def collect_match_stats(match_ids, api_key):
    all_rows = []
    for match_id in match_ids:
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {
            "X-Riot-Token": api_key
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            match_data = response.json()
            rows = parse_match_data(match_data)
            all_rows.extend(rows)
        else:
            print(f"Failed to retrieve match {match_id}: {response.status_code}")
    return pd.DataFrame(all_rows)


In [5]:
df = collect_match_stats(match_ids, api_key=API_KEY)

In [6]:
df

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,wardsPlaced,goldDiff,allInPings,assistMePings,commandPings,enemyMissingPings,enemyVisionPings,holdPings,getBackPings,totalPings
0,NA1_5279161872,100,True,1,3,0,73556,66450,58,41,...,52,9080,2,10,7,2,5,0,1,27
1,NA1_5279161872,200,False,0,2,1,64476,58900,41,58,...,57,-9080,6,7,15,13,1,0,5,47
2,NA1_5277488964,100,True,0,3,1,42741,36688,27,13,...,40,5008,4,7,16,41,17,0,3,88
3,NA1_5277488964,200,False,0,0,0,37733,35458,13,27,...,36,-5008,2,9,37,35,10,0,1,94
